Montar Entorno

In [0]:
!nvcc --version
!python --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Python 3.6.8


In [0]:
#!pip install python 3.7.1
!pip install scipy
#!pip install turicreate==5.8

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Traer Archivo Dataset

In [0]:
data_path = '/content/dataset/'
data_path_train =  '/content/dataset/train/'
data_path_test =   '/content/dataset/test/'
data_path_valid =  '/content/dataset/valid/'
#save_path = f'/content/drive/My Drive/Saturdays.AI/Proyecto1/'
save_path = f'/content/'
drive_path = "/content/drive/My Drive/Saturdays.AI/Proyecto1/"

In [0]:
import tarfile
#del os.environ['LC_ALL']
#Traemos el dataset
tar_file = "datasetBasura.tar.gz"
data_zip = f"/content/drive/My\ Drive/Saturdays.AI/Proyecto1/{tar_file}"
dest = '/content/'
data_file = f'/content/{tar_file}'

!rm -rf {data_path} # Borramos archivos previos #!rm -rf basura/
!rm -rf sample_data/

!cp -a {data_zip} {dest} # copiar archivo desde Drive

tar = tarfile.open(data_file) # descomprimir
tar.extractall()
tar.close()
print("Archivo descomprimido...")

!rm {data_file} # borrar lo que no se ocupa

# borramos todos los archivos basura que genera MacOS
print("Borrando archivos basura...")
!find /content/dataset/ -name ".*" -exec rm -rf {} \;
print("Archivos basura eliminados...")

Archivo descomprimido...
Borrando archivos basura...
Archivos basura eliminados...


#Aumento de Data

In [0]:
cant_archivos_x_imagen = 3
cant_transformaciones_x_imagen = 4
max_repeticiones_transformaciones = 2

In [0]:
import os
import ntpath
import random
import math
import numpy as np
from scipy import ndarray

# image processing library
import skimage as sk
from skimage import transform
from skimage import util
from skimage import io

###############################################################################
####                    FUNCIONES DE TRANSFORMACION                        ####
###############################################################################
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-180, 180)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

def swirl(image_array: ndarray):
    radio = random.uniform(100,1000)
    fuerza = (3 * (1-(radio/1000)))
    #print(f"Radio:{radio} , Fuerza:{fuerza}")
    centro = [random.uniform(0, image_array.shape[0]), random.uniform(0, image_array.shape[1])]
    #return sk.transform.swirl(image_array, center=centro, strength=100, radius=600, rotation=random.uniform(-180,180))
    return sk.transform.swirl(image_array, center=centro, strength=fuerza, radius=radio, rotation=0)

def wave(image_array: ndarray):
    rows, cols = image_array.shape[0], image_array.shape[1]
    src_cols = np.linspace(0, cols, 20)
    src_rows = np.linspace(0, rows, 10)
    src_rows, src_cols = np.meshgrid(src_rows, src_cols)
    src = np.dstack([src_cols.flat, src_rows.flat])[0]

    # add sinusoidal oscillation to row coordinates
    dst_rows = src[:, 1] - np.sin(np.linspace(0, 3 * np.pi, src.shape[0])) * 25
    dst_cols = src[:, 0]
    #dst_rows *= 1.5
    #dst_rows -= 1.5 * 15
    dst = np.vstack([dst_cols, dst_rows]).T
    tform = sk.transform.PiecewiseAffineTransform()
    tform.estimate(src, dst)
    out_rows = image_array.shape[0] - 1.5 * 0
    out_cols = cols
    return sk.transform.warp(image_array, tform, output_shape=(out_rows, out_cols))

def rescale(image_array: ndarray):
    escala = (random.uniform(0.5, 1), random.uniform(0.6, 1));
    return sk.transform.rescale(image_array, escala, anti_aliasing=True)  

# dictionary of the transformations we defined earlier
available_transformations = {
    'rotate1': random_rotation,
    'rotate2': random_rotation,
    'rotate3': random_rotation,
    'noise1': random_noise,
    'noise2': random_noise,
    'horizontal_flip1': horizontal_flip,
    'horizontal_flip2': horizontal_flip,
    'swirl': swirl,
    'wave': wave,
    "rescale1": rescale,
    "rescale2": rescale
}

###############################################################################

image_files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(data_path_train):
    for file in f:
        #if '.txt' in file:
            image_files.append(os.path.join(r, file))

for imagen in image_files:
    archivos_creados = 0
  
    # Crear n archivo por cada imagen
    while archivos_creados < cant_archivos_x_imagen:
        image_to_transformm = sk.io.imread(imagen)
        transformaciones_aplicadas = 0
        tipo_transformaciones_aplicadas = { 'None' : -1}
        
        while transformaciones_aplicadas < cant_transformaciones_x_imagen:
            if(transformaciones_aplicadas >= 1 and random.uniform(0,1) < 0.7):
                break
            # la transformacion a aplicar
            key = random.choice(list(available_transformations))
            if key not in tipo_transformaciones_aplicadas:
                tipo_transformaciones_aplicadas[key] = 0
            # Que no hayamos excedido la repeticiones de transformacion
            if(tipo_transformaciones_aplicadas[key] < max_repeticiones_transformaciones):
                image_to_transform = available_transformations[key](image_to_transformm)
                base_archivo, extension = ntpath.splitext(imagen)
                #new_file_path = f"{dest_path}{ntpath.basename(base_archivo)}_{archivos_creados}{extension}"
                new_file_path = f"{ntpath.dirname(base_archivo)}/{ntpath.basename(base_archivo)}_{archivos_creados}{extension}"
                
            tipo_transformaciones_aplicadas[key] += 1;
            transformaciones_aplicadas += 1
        
        # salvamos el archivo
        io.imsave(new_file_path, image_to_transform)
        archivos_creados += 1

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


In [0]:
!pip install turicreate==5.8

     |████████████████████████████████| 90.7MB 81kB/s 
     |████████████████████████████████| 3.4MB 36.1MB/s 
     |████████████████████████████████| 2.1MB 43.8MB/s 
     |████████████████████████████████| 17.3MB 142kB/s 
     |████████████████████████████████| 327kB 42.8MB/s 
     |████████████████████████████████| 23.8MB 1.2MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320847 sha256=aad5f3b18d32829aa3aea43f47b2043f044f19b025a6829bc927c7699b9c7b11
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requi

In [0]:
import turicreate as tc
#train_data = tc.image_analysis.load_images("/content/drive/My Drive/Saturdays.AI/TuriCreate/Snacks/train", with_path=True)
train_data = tc.image_analysis.load_images(data_path_train, with_path=True)
len(train_data)

1482

In [0]:
#test_data = tc.image_analysis.load_images("/content/drive/My Drive/Saturdays.AI/TuriCreate/Snacks/test", with_path=True)
test_data = tc.image_analysis.load_images(data_path_test, with_path=True)
len(test_data)

431

In [0]:
import os  #ya se importO
train_data["label"] = train_data["path"].apply(lambda path: 
                                               os.path.basename(os.path.split(path)[0]))
test_data["label"] = test_data["path"].apply(lambda path: 
                                             os.path.basename(os.path.split(path)[0]))
train_data["label"].value_counts().print_rows(num_rows=20)
test_data["label"].value_counts().print_rows(num_rows=20)

+--------------+-------+
|    value     | count |
+--------------+-------+
|    vidrio    |  354  |
|   plastico   |  347  |
|    metal     |  286  |
|    papel     |  204  |
|    carton    |  200  |
| noReciclable |   91  |
+--------------+-------+
[6 rows x 2 columns]

+--------------+-------+
|    value     | count |
+--------------+-------+
|    Papel     |  108  |
|    Vidrio    |   82  |
|   Plastico   |   74  |
|    Carton    |   70  |
|    Metal     |   68  |
| noReciclable |   29  |
+--------------+-------+
[6 rows x 2 columns]



/usr/local/lib/python3.6/dist-packages/turicreate/aggregate.py:92: UserWarning: Passing parameter(s) to COUNT is deprecated. This functionality will be removed in the next major release. Any passed parameters are ignored.
  + 'the next major release. Any passed parameters are ignored.')


Run the next cell to train the model.

**OR**

Run the cell **after** the next cell, to load the model from the `starter/notebook` folder.

In [0]:
# Train the image classifier
model = tc.image_classifier.create(train_data, target="label", 
                                   model="squeezenet_v1.1",
                                   verbose=True, max_iterations=1000)

#100 iterations
  #Accuracy:  0.7749419953596288
  #Precision:  0.7532829154205966
  #Recall:  0.7467164667217437

#100 iteraciones New
  #Accuracy:  0.7911832946635731
  #Precision:  0.7680419616694126
  #Recall:  0.7622161537422342

#200 iteraciones
  #Accuracy:  0.7795823665893271
  #Precision:  0.7651109840859158
  #Recall:  0.745827363232416

#200 iteraciones New
  #Accuracy:  0.7679814385150812
  #Precision:  0.7449941017879439
  #Recall:  0.7343972031413282

#300 iteraciones
  #Accuracy:  0.7749419953596288
  #Precision:  0.7607788355029602
  #Recall:  0.7495416116019192

#400 iteraciones
  #Accuracy:  0.7935034802784223
  #Precision:  0.7811909479644243
  #Recall:  0.767284889483887

#500 iteraciones
  #Accuracy:  0.7911832946635731
  #Precision:  0.7795764544268543
  #Recall:  0.764883755686292

#500 iteraciones Desconocido
  #Accuracy:  0.11578947368421053
  #Precision:  0.2383882783882784
  #Recall:  0.2087998208687864

#700 iteraciones
  #Accuracy:  0.7911832946635731
  #Precision:  0.7678162947876581
  #Recall:  0.7599331284199934

#1000 iteraciones
  #Accuracy:  0.777262180974478
  #Precision:  0.7638553150413875
  #Recall:  0.7482981672991403

#1000 iteraciones New
  #Accuracy:  0.802784222737819
  #Precision:  0.7804722320824832
  #Recall:  0.7669611392141448

Performing feature extraction on resized images...
Completed   64/1482
Completed  128/1482
Completed  192/1482
Completed  256/1482
Completed  320/1482
Completed  384/1482
Completed  448/1482
Completed  512/1482
Completed  576/1482
Completed  640/1482
Completed  704/1482
Completed  768/1482
Completed  832/1482
Completed  896/1482
Completed  960/1482
Completed 1024/1482
Completed 1088/1482
Completed 1152/1482
Completed 1216/1482
Completed 1280/1482
Completed 1344/1482
Completed 1408/1482
Completed 1472/1482
Completed 1482/1482
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 1407

Number of classes           : 6

...

| 424       | 658      | 1.000000  | 66.902955    | 0.999289          | 0.826667            |

| 449       | 692      | 1.000000  | 70.515972    | 0.999289          | 0.826667            |

| 474       | 729      | 1.000000  | 74.304933    | 0.999289          | 0.853333            |

| 499       | 764      | 0.191785  | 77.955934    | 0.999289          | 0.880000            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

#Salvar y Exportar Modelo

In [0]:
!rm -rf ./basura.model
!rm -f BasuraModel.tar.gz
model.save(f"{save_path}Basura.model")
model.save(f"{drive_path}Basura.model")
model.export_coreml(f"{save_path}Basura.mlmodel")
model.export_coreml(f"{drive_path}Basura.mlmodel")
!tar -czf BasuraModel.tar.gz Basura.model/
#model.export_coreml(f"{save_path}basura.mlmodel")

/usr/local/lib/python3.6/dist-packages/_pytest/mark/structures.py:426: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)
/usr/local/lib/python3.6/dist-packages/coremltools/_deps/__init__.py:71: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  % (tensorflow.__version__, TF_MAX_VERSION))
/usr/local/lib/python3.6/dist-packages/coremltools/_deps/__init__.py:117: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  % (keras.__version__, KERAS_MAX_VERSION))


In [0]:
# OR load HealthySnacks.model from starter/notebook folder
model = tc.load_model(f"{save_path}Basura.model")

Continue with trained or loaded model.

In [0]:
metrics = model.evaluate(test_data)
print("Accuracy: ", metrics["accuracy"])
print("Precision: ", metrics["precision"])
print("Recall: ", metrics["recall"])

Performing feature extraction on resized images...
Completed  64/431
Completed 128/431
Completed 192/431
Completed 256/431
Completed 320/431
Completed 384/431
Completed 431/431


/usr/local/lib/python3.6/dist-packages/turicreate/aggregate.py:92: UserWarning: Passing parameter(s) to COUNT is deprecated. This functionality will be removed in the next major release. Any passed parameters are ignored.
  + 'the next major release. Any passed parameters are ignored.')


Accuracy:  0.7911832946635731
Precision:  0.7734444161606441
Recall:  0.7694097918656371


In [0]:
model.predict(test_data)

In [0]:
output = model.classify(test_data)
output

In [0]:
images_with_prediction = test_data.add_columns(output)
images_with_prediction.explore()

In [0]:
images_filtered = images_with_prediction[
    (images_with_prediction["probability"] > 0.9) &
    (images_with_prediction["label"]) != images_with_prediction["class"]
]
images_filtered.explore()

In [0]:
predictions = model.predict(test_data, output_type='probability_vector')

In [0]:
predictions[0]

In [0]:
labels = test_data["label"].unique().sort()
preds = tc.SArray(predictions[0])
tc.SFrame()